<a href="https://colab.research.google.com/github/ArnaudOlt/dissertation_pybamm/blob/main/5_1_SOC_degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pybtex
%pip install anytree
%pip install autograd
%pip install bpx
%pip install casadi
%pip install imageio
%pip install importlib-metadata
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install scikit-fem
%pip install scipy
%pip install sympy
%pip install tqdm
%pip install xarray
%pip install git+https://github.com/pybamm-team/pybamm.git@develop
import pybamm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
prada2013_params = pybamm.ParameterValues("Prada2013")

def plating_exchange_current_density_OKane2020(c_e, c_Li, T):
    """
    Exchange-current density for Li plating reaction [A.m-2].
    References
    ----------
    .. [1] O’Kane, Simon EJ, Ian D. Campbell, Mohamed WJ Marzook, Gregory J. Offer, and
    Monica Marinescu. "Physical origin of the differential voltage minimum associated
    with lithium plating in Li-ion batteries." Journal of The Electrochemical Society
    167, no. 9 (2020): 090540.
    Parameters
    ----------
    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_Li : :class:`pybamm.Symbol`
        Plated lithium concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]
    Returns
    -------
    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    k_plating = pybamm.Parameter("Lithium plating kinetic rate constant [m.s-1]")

    return pybamm.constants.F * k_plating * c_e


def stripping_exchange_current_density_OKane2020(c_e, c_Li, T):
    """
    Exchange-current density for Li stripping reaction [A.m-2].

    References
    ----------

    .. [1] O’Kane, Simon EJ, Ian D. Campbell, Mohamed WJ Marzook, Gregory J. Offer, and
    Monica Marinescu. "Physical origin of the differential voltage minimum associated
    with lithium plating in Li-ion batteries." Journal of The Electrochemical Society
    167, no. 9 (2020): 090540.

    Parameters
    ----------

    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_Li : :class:`pybamm.Symbol`
        Plated lithium concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]

    Returns
    -------

    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    k_plating = pybamm.Parameter("Lithium plating kinetic rate constant [m.s-1]")

    return pybamm.constants.F * k_plating * c_Li

def SEI_limited_dead_lithium_OKane2022(L_sei):
    """
    Decay rate for dead lithium formation [s-1].
    References
    ----------
    .. [1] Simon E. J. O'Kane, Weilong Ai, Ganesh Madabattula, Diega Alonso-Alvarez,
    Robert Timms, Valentin Sulzer, Jaqueline Sophie Edge, Billy Wu, Gregory J. Offer
    and Monica Marinescu. "Lithium-ion battery degradation: how to model it."
    Physical Chemistry: Chemical Physics 24, no. 13 (2022): 7909-7922.
    Parameters
    ----------
    L_sei : :class:`pybamm.Symbol`
        Total SEI thickness [m]
    Returns
    -------
    :class:`pybamm.Symbol`
        Dead lithium decay rate [s-1]
    """

    gamma_0 = pybamm.Parameter("Dead lithium decay constant [s-1]")
    L_inner_0 = pybamm.Parameter("Initial inner SEI thickness [m]")
    L_outer_0 = pybamm.Parameter("Initial outer SEI thickness [m]")
    L_sei_0 = L_inner_0 + L_outer_0

    gamma = gamma_0 * L_sei_0 / L_sei

    return gamma



prada2013_params.update({
        #experiment
        "Lower voltage cut-off [V]": 2.5,
        "Upper voltage cut-off [V]": 3.65,
        #cell
        "Negative current collector thickness [m]": 0.000006,
        "Separator thickness [m]": 0.000016,
        "Positive current collector thickness [m]": 0.000013,
        "Electrode height [m]": 0.325,
        "Electrode width [m]": 21.6,
        "Cell cooling surface area [m2]": 0.07338004,
        "Cell volume [m3]": 0.00106398976,
        "Positive current collector density [kg.m-3]": 2700,
        "Positive current collector thermal conductivity [W.m-1.K-1]": 237,
        "Nominal cell capacity [A.h]":100,
        "Current function [A]":100,
        "Contact resistance [Ohm]": 0.0004,
        #electrolyte
        "Electrolyte conductivity [S.m-1]": 0.97,



        #Okane2022 Li plating
        "Lithium metal partial molar volume [m3.mol-1]": 1.3e-05,
        "Lithium plating kinetic rate constant [m.s-1]": 1e-09,
        "Exchange-current density for plating [A.m-2]": plating_exchange_current_density_OKane2020,
        "Exchange-current density for stripping [A.m-2]": stripping_exchange_current_density_OKane2020,
        "Initial plated lithium concentration [mol.m-3]": 0.0,
        "Typical plated lithium concentration [mol.m-3]": 1000.0,
        "Lithium plating transfer coefficient": 0.65,
        "Dead lithium decay constant [s-1]": 1e-06,
        "Dead lithium decay rate [s-1]": SEI_limited_dead_lithium_OKane2022,
        #Okane2022 SEI
        "Ratio of lithium moles to SEI moles": 1.0,
        "Inner SEI reaction proportion": 0.0,
        "Inner SEI partial molar volume [m3.mol-1]": 9.585e-05,
        "Outer SEI partial molar volume [m3.mol-1]": 9.585e-05,
        "SEI reaction exchange current density [A.m-2]": 1.5e-07,
        "SEI resistivity [Ohm.m]": 200000.0,
        "Outer SEI solvent diffusivity [m2.s-1]": 2.5000000000000002e-22,
        "Bulk solvent concentration [mol.m-3]": 2636.0,
        "Inner SEI open-circuit potential [V]": 0.1,
        "Outer SEI open-circuit potential [V]": 0.8,
        "Inner SEI electron conductivity [S.m-1]": 8.95e-14,
        "Inner SEI lithium interstitial diffusivity [m2.s-1]": 1e-20,
        "Lithium interstitial reference concentration [mol.m-3]": 15.0,
        "Initial inner SEI thickness [m]": 0.0,
        "Initial outer SEI thickness [m]": 5e-09,
        "EC initial concentration in electrolyte [mol.m-3]": 4541.0,
        "EC diffusivity [m2.s-1]": 2e-18,
        "SEI kinetic rate constant [m.s-1]": 1e-12,
        "SEI open-circuit potential [V]": 0.4,
        "SEI growth activation energy [J.mol-1]": 38000.0,
        "Negative electrode reaction-driven LAM factor [m3.mol-1]": 0.0,
        "Positive electrode reaction-driven LAM factor [m3.mol-1]": 0.0

        }, check_already_exists=False)


In [ ]:
# Scenario HIGH

pybamm.set_logging_level("NOTICE")

T_ambs = [-5, 5, 15] #,15

# Number of cycles per temperature setting
N_cycles_per_param = 500
# Done to reduce RAM usage, clears RAM after every slice of N cycles
cycles_per_slice = 40

# C-rates
charging_C_rate = 0.08
discharging_C_rate = 0.02

# Which initial SoC's to cycle between
lower_SoC = 0.7
upper_SoC = 0.99

# Setting submodel options:
li_plt_options = {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
    }




In [ ]:
solver = pybamm.CasadiSolver(atol = 1e-6, rtol = 1e-6, mode="safe", dt_max=6)

In [ ]:
# Making the mesh finer to avoid Solver Errors
var_pts = {
    "x_n": 50,  # negative electrode
    "x_s": 50,  # separator
    "x_p": 50,  # positive electrode
    "r_n": 50,  # negative particle
    "r_p": 50,  # positive particle
}

In [ ]:
# Running an experiment to get the initial battery cell capacity

mapping_experiment = pybamm.Experiment([
    ("Rest for 1 minute")
])

In [ ]:
# extracting initial capacity of a fresh LiB cell by solving
# the model and reading the "Capacity [A.h]" summery variable
SPMe = pybamm.lithium_ion.SPMe()
mapping_sim = pybamm.Simulation(SPMe, parameter_values=prada2013_params, solver=solver, experiment=mapping_experiment, var_pts=var_pts)
# Start at 0.3 SoC to avoid minimu/maximum voltage error
mapping_sol = mapping_sim.solve(initial_soc=lower_SoC)
mapping_solution = mapping_sim.solution
initial_cap = mapping_solution.summary_variables["Capacity [A.h]"][0] #Ah

In [ ]:
def sim_N_cycles_individually(N_cycles, N_per_slice, solution, lower_SoC, upper_SoC, options, params, solver, var_pts, C_charge, C_discharge):
  sim_stopped_progression = False
  # cycling through every cycle one by one
  for cycle_N in range(1, N_cycles + 1):
    #optaining remaining capacity
    remaining_cap = solution.summary_variables["Capacity [A.h]"][-1]
    initial_lower_Q = lower_SoC * remaining_cap # Ah
    initial_upper_Q = upper_SoC * remaining_cap # Ah
    charging_current = C_charge * remaining_cap # A
    discharging_current = C_discharge * remaining_cap # A
    charge_time = (initial_upper_Q - initial_lower_Q)/charging_current # h
    discharge_time = (initial_upper_Q - initial_lower_Q)/discharging_current # h

    # defining experiment, model and simulation
    experiment_N_1 = pybamm.Experiment([
        (f"Charge at {charging_current} A for {charge_time} hours",
         f"Discharge at {discharging_current} A for {discharge_time} hours")
    ])

    SPMe = pybamm.lithium_ion.SPMe(options=options)
    sim = pybamm.Simulation(SPMe, parameter_values=prada2013_params, solver = solver, experiment=experiment_N_1, var_pts = var_pts)

    if cycle_N == 1:
      # solving sim, first cycle starting on lower SoC
      sol = sim.solve(initial_soc=lower_SoC)
      try:
        len(sol.cycles)
      except AttributeError:
        raise Exception ("The first cycle did not complete")
    else:
      # solving sim, startin on sol
      sol = sim.solve(starting_solution = sol)
    # obtaining infor for startin point for next cycle
    solution = sim.solution

    #check if last cycle were successfully solved
    if len(sol.cycles) != cycle_N:
      sim_stopped_progression = True


    if cycle_N%N_per_slice == 0 or sim_stopped_progression == True:
      if sim_stopped_progression:
        return [solution,sol]
  return [solution, sol]

In [ ]:
# SIM start, one for each parameter value
solutions_70_99 = {}
for T_amb in T_ambs:
    #updating the param which is investigated
    prada2013_params.update({"Ambient temperature [K]": 273.15 + T_amb})
    conductSim = sim_N_cycles_individually(N_cycles = N_cycles_per_param, N_per_slice = cycles_per_slice, solution = mapping_solution, lower_SoC = lower_SoC, upper_SoC = upper_SoC, options = li_plt_options, params = prada2013_params, solver = solver, var_pts = var_pts, C_charge = charging_C_rate, C_discharge = discharging_C_rate)
    solutions_70_99[T_amb] = conductSim
    # Saving specified data
    #save_data_to_csv(N_per_slice = cycles_per_slice, solution = conductSim[0], file_name_add_on = str(T_amb))
    #solutions_70_99[T_amb][0].save(f"solutions_70_99_solution_{T_amb}.pkl")
    solutions_70_99[T_amb][1].save(f"solutions_70_99_lunana_C-rate_sol_{T_amb}.pkl")

In [ ]:
import matplotlib.pyplot as plt

x_axis_var_str = "Cycle number"
y_axis_var_str = "Measured capacity [A.h]"

fig, ax = plt.subplots(figsize=(8, 6))

# Specify colors for each line
line_colors = ["steelblue","lightblue", "coral", "coral", "cyan", "magenta", "yellow"]

for i, (temp, sol_list) in enumerate(solutions_70_99.items()):
    sol = sol_list[1]  # Access the first element in the list
    x_sol = sol.summary_variables[x_axis_var_str]
    y_sol = sol.summary_variables[y_axis_var_str]

    color = line_colors[i % len(line_colors)]  # Cycle through colors if needed

    ax.plot(
        x_sol,
        ((y_sol[0] - y_sol) / y_sol[0]) * 100,
        label="Temperature = " + str(temp) + " °C",
        color=color,
    )

ax.set_xlabel(x_axis_var_str)
ax.set_ylabel("Capacity loss [%]")
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Scenario low

pybamm.set_logging_level("NOTICE")

T_ambs = [-5,5,15]

# Number of cycles per temperature setting
N_cycles_per_param = 500
# Done to reduce RAM usage, clears RAM after every slice of N cycles
cycles_per_slice = 40

# C-rates
charging_C_rate = 1
discharging_C_rate = 0.3

# Which initial SoC's to cycle between
lower_SoC = 0.3
upper_SoC = 0.6

# Setting submodel options:
li_plt_options = {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
    }




In [ ]:
solver = pybamm.CasadiSolver(atol = 1e-6, rtol = 1e-6, mode="safe", dt_max=6)

In [ ]:
# Making the mesh finer to avoid Solver Errors
var_pts = {
    "x_n": 50,  # negative electrode
    "x_s": 50,  # separator
    "x_p": 50,  # positive electrode
    "r_n": 50,  # negative particle
    "r_p": 50,  # positive particle
}

In [ ]:
# Running an experiment to get the initial battery cell capacity

mapping_experiment = pybamm.Experiment([
    ("Rest for 1 minute")
])

In [ ]:
# extracting initial capacity of a fresh LiB cell by solving
# the model and reading the "Capacity [A.h]" summery variable
SPMe = pybamm.lithium_ion.SPMe()
mapping_sim = pybamm.Simulation(SPMe, parameter_values=prada2013_params, solver=solver, experiment=mapping_experiment, var_pts=var_pts)
# Start at 0.3 SoC to avoid minimu/maximum voltage error
mapping_sol = mapping_sim.solve(initial_soc=lower_SoC)
mapping_solution = mapping_sim.solution
initial_cap = mapping_solution.summary_variables["Capacity [A.h]"][0] #Ah

In [ ]:
def sim_N_cycles_individually(N_cycles, N_per_slice, solution, lower_SoC, upper_SoC, options, params, solver, var_pts, C_charge, C_discharge):
    sim_stopped_progression = False
    # cycling through every cycle one by one
    for cycle_N in range(1, N_cycles + 1):
        #optaining remaining capacity
        remaining_cap = solution.summary_variables["Capacity [A.h]"][-1]
        # play if measured capacity and capacity
        initial_lower_Q = lower_SoC * remaining_cap # Ah
        initial_upper_Q = upper_SoC * remaining_cap # Ah
        charging_current = C_charge * remaining_cap # A
        discharging_current = C_discharge * remaining_cap # A
        charge_time = (initial_upper_Q - initial_lower_Q)/charging_current # h
        discharge_time = (initial_upper_Q - initial_lower_Q)/discharging_current # h

        # defining experiment, model and simulation
        experiment_N_1 = pybamm.Experiment([
            (f"Charge at {charging_current} A for {charge_time} hours",
             f"Discharge at {discharging_current} A for {discharge_time} hours")
        ])

        SPMe = pybamm.lithium_ion.SPMe(options=options)
        sim = pybamm.Simulation(SPMe, parameter_values=prada2013_params, solver = solver, experiment=experiment_N_1, var_pts = var_pts)

        if cycle_N == 1:
            # solving sim, first cycle starting on lower SoC
            sol = sim.solve(initial_soc=lower_SoC, save_at_cycles = 5)
            try:
                len(sol.cycles)
            except AttributeError:
                raise Exception ("The first cycle did not complete")
        else:
          # solving sim, startin on sol
          sol = sim.solve(starting_solution = sol, save_at_cycles = 5)
        # obtaining infor for startin point for next cycle
        solution = sim.solution

        #check if last cycle were successfully solved
        if len(sol.cycles) != cycle_N:
            sim_stopped_progression = True


        if cycle_N%N_per_slice == 0 or sim_stopped_progression == True:
            if sim_stopped_progression:
                return [solution,sol]
    return [solution, sol]

In [ ]:
# SIM start, one for each parameter value
solutions_30_60 = {}
for T_amb in T_ambs:
    #updating the param which is investigated
    prada2013_params.update({"Ambient temperature [K]": 273.15 + T_amb})
    conductSim = sim_N_cycles_individually(N_cycles = N_cycles_per_param, N_per_slice = cycles_per_slice, solution = mapping_solution, lower_SoC = lower_SoC, upper_SoC = upper_SoC, options = li_plt_options, params = prada2013_params, solver = solver, var_pts = var_pts, C_charge = charging_C_rate, C_discharge = discharging_C_rate)
    solutions_30_60[T_amb] = conductSim
    # Saving specified data
    #save_data_to_csv(N_per_slice = cycles_per_slice, solution = conductSim[0], file_name_add_on = str(T_amb))
    #solutions_30_60[T_amb][0].save(f"solutions_30_60_solution_{T_amb}.pkl")
    solutions_30_60[T_amb][1].save(f"solutions_30_60_sol_{T_amb}.pkl")
